### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        self.imp = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.imp = self.__gini_imp
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.imp = self.__entropy_imp
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.imp = self.__misclass_imp
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        return (l_s*(1-np.sum((l_c/l_s)**2, axis=1)).reshape(-1, 1) +
                r_s*(1-np.sum((r_c/r_s)**2, axis=1)).reshape(-1, 1)) / (l_s+r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return (l_s * (-np.sum((l_c/l_s) * np.log(l_c/l_s), axis=1)).reshape(-1, 1) +
                r_s * (-np.sum((r_c/r_s) * np.log(r_c/r_s), axis=1)).reshape(-1, 1)) / (l_s+r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (l_s * (1 - np.max(l_c/l_s, axis=1)).reshape(-1, 1) +
                r_s * (1 - np.max(r_c/r_s, axis=1)).reshape(-1, 1)) / (l_s+r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.round(np.sqrt(n_features)):]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.round(np.log2(n_features)):]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __gini_imp(self, y):
        return 1 - ((np.bincount(y).astype('float') / y.size) ** 2).sum()
    
    def __entropy_imp(self, y):
        bins = np.bincount(y).astype('float') / y.size
        return -np.nansum(bins * np.log2(bins))
    
    def __misclass_imp(self, y):
        return 1 - np.max(np.bincount(y).astype('float') / y.size)

    def __find_threshold(self, x, y):
        x_sort, y_sort = self.__sort_samples(x, y)

        class_brd = np.where(y_sort[:-1] != y_sort[1:])[0] + 1

        if len(class_brd) == 0:
            return np.inf, None

        eq_el_count = class_brd - np.append([0], class_brd[:-1])
        one_hot_code = np.zeros((class_brd.shape[0], self.num_class))
        one_hot_code[np.arange(class_brd.shape[0]), y_sort[class_brd-1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)

        l_c = np.cumsum(class_increments, axis=0)
        r_c = np.bincount(y, minlength=self.num_class) - l_c
        l_s = class_brd.reshape(-1, 1)
        r_s = y_sort.shape[0] - l_s

        gs = self.G_function(l_c, l_s, r_c, r_s)
        idx = np.argmin(gs)

        left_el_id = l_s[idx][0]
        threshold = (x_sort[left_el_id -1] + x_sort[left_el_id ]) / 2.0
        return gs[idx], threshold

    def __fit_node(self, x, y, node_id, depth):
        count = np.bincount(y)
        if self.max_depth is not None and depth == self.max_depth or\
                    y.shape[0] < self.min_samples_split or\
                    self.sufficient_share <= np.float(np.bincount(y).max()) / y.size:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(count), np.array(count, dtype=np.float) / y.size)
            return
        indices = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, idx], y) for idx in indices])
        min_thr = thresholds[:, 0].argmin()
        optimal_threshold = thresholds[min_thr, 1]

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, indices[min_thr], optimal_threshold)
        
        if x_l.shape[0] == 0 or x_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, count.argmax(), count.astype('float')/y.size)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, indices[min_thr], optimal_threshold)

        self.feature_importances_[min_thr] += y.size*self.imp(y) - y_l.size*self.imp(y_l) - y_r.size*self.imp(y_r)

        self.__fit_node(x_l, y_l, 2*node_id + 1, depth+1)
        self.__fit_node(x_r, y_r, 2*node_id + 2, depth+1)
    
    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 22 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 81 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8363636363636364

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8307692307692308

## Подготовка данных Speed Dating Data 

In [9]:
import pandas as pd
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')

print(df.head())

df = df.iloc[:, :97]

droplist = ['id', 
            'idg', 'condtn', 'round', 'position', 'positin1',
            'order', 'partner', 'age_o', 'race_o', 'pf_o_att',
            'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 
            'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o', 
            'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 
            'met_o', 'field', 'undergra', 'from', 'zipcode', 
            'career', 'sports', 'tvsports', 'exercise', 'dining', 
            'museums', 'art', 'hiking', 'gaming', 'clubbing', 
            'reading', 'tv', 'theater', 'movies', 'concerts',
            'music', 'shopping', 'yoga', 'expnum', 'wave']

df = df.drop(droplist, axis=1)


dropnalist = ['age', 'imprelig', 'imprace', 'date']
df.dropna(subset=dropnalist)

pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')

df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)


buf = ['field_cd', 'mn_sat', 'tuition', 'income', 'career_c']
df.loc[:, buf] = df.loc[:, buf].fillna(0)



df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                        'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1',
           'intel1_1', 'fun1_1',
           'amb1_1', 'shar1_1']] = (
    df.loc[:, ['attr1_1', 'sinc1_1',
                'intel1_1', 'fun1_1',
                'amb1_1', 'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1',
           'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = (
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)


for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)


df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']
                                                 ).drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']
                                                   ).drop(['gender', 'match', 'int_corr', 'samerace'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X, y

   iid   id  gender  idg  condtn  wave  round  position  positin1  order  ...  \
0    1  1.0       0    1       1     1     10         7       NaN      4  ...   
1    1  1.0       0    1       1     1     10         7       NaN      3  ...   
2    1  1.0       0    1       1     1     10         7       NaN     10  ...   
3    1  1.0       0    1       1     1     10         7       NaN      5  ...   
4    1  1.0       0    1       1     1     10         7       NaN      7  ...   

   attr3_3  sinc3_3  intel3_3  fun3_3  amb3_3  attr5_3  sinc5_3  intel5_3  \
0      5.0      7.0       7.0     7.0     7.0      NaN      NaN       NaN   
1      5.0      7.0       7.0     7.0     7.0      NaN      NaN       NaN   
2      5.0      7.0       7.0     7.0     7.0      NaN      NaN       NaN   
3      5.0      7.0       7.0     7.0     7.0      NaN      NaN       NaN   
4      5.0      7.0       7.0     7.0     7.0      NaN      NaN       NaN   

   fun5_3  amb5_3  
0     NaN     NaN  
1     NaN 

(array([[ 1.4e-01,  0.0e+00,  2.7e+01, ...,  8.0e+00,  8.0e+00,  7.0e+00],
        [ 5.4e-01,  0.0e+00,  2.2e+01, ...,  8.0e+00,  8.0e+00,  7.0e+00],
        [ 1.6e-01,  1.0e+00,  2.2e+01, ...,  8.0e+00,  8.0e+00,  7.0e+00],
        ...,
        [-2.0e-01,  0.0e+00,  3.0e+01, ...,  9.0e+00,  9.0e+00,  7.0e+00],
        [-3.2e-01,  0.0e+00,  2.7e+01, ...,  9.0e+00,  9.0e+00,  7.0e+00],
        [ 1.0e-02,  0.0e+00,  2.5e+01, ...,  9.0e+00,  9.0e+00,  7.0e+00]]),
 array([0, 0, 1, ..., 0, 0, 0], dtype=int64))

## Проверка скорости работы на Speed Dating Data 

In [10]:
%time clf.fit(X_train, y_train)

Wall time: 500 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [11]:
%time my_clf.fit(X_train, y_train)

Wall time: 2.56 s


## Проверка качества работы на Speed Dating Data

In [12]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5135548165606604

In [13]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.45725915875169604

## Задание 3

In [14]:
idx = np.argsort(clf.feature_importances_)[::-1][:10]
clf_imp = df_pair.columns[1:][idx]
my_idx = np.argsort(my_clf.feature_importances_)[::-1][:10]
my_clf_imp = df_pair.columns[1:][my_idx]

importances = pd.DataFrame(columns=['DecisionTreeClassifier', 'clfImportance',
                                   'MyDecisionTreeClassifier', 'my_clfImportance'])
importances['MyDecisionTreeClassifier'] = my_clf_imp
importances['DecisionTreeClassifier'] = clf_imp
importances['clfImportance'] = clf.feature_importances_[idx]
importances['my_clfImportance'] = my_clf.feature_importances_[my_idx]

## Задание 4

In [15]:
importances

,DecisionTreeClassifier,clfImportance,MyDecisionTreeClassifier,my_clfImportance
0,int_corr,0.076462,int_corr,0.092607
1,sinc1_1,0.031170,date,0.082310
2,amb1_1,0.025885,amb1_1_f,0.071761
3,intel2_1_f,0.025299,attr2_1,0.069681
4,sinc1_1_f,0.025098,fun3_1,0.061780
5,sinc2_1_f,0.024221,samerace,0.059123
6,age_f,0.023978,career_c,0.054102
7,attr1_1,0.023955,age,0.052412
8,attr3_1_f,0.023356,income_f,0.049963
9,sinc3_1,0.022698,imprelig_f,0.049879


## Задание 5

In [16]:
params = {
    'max_depth': range(1, 15),
    'min_samples_leaf': range(1, 20),
    'criterion': ['gini', 'entropy'],
    'n_estimators': range(1, 100, 10)
}
forest = RandomForestClassifier()
optimal_clf = RandomizedSearchCV(forest, param_distributions=params, cv=5)
optimal_clf.fit(X_train, y_train)
optimal_clf.best_params_

{'n_estimators': 31,
 'min_samples_leaf': 1,
 'max_depth': 13,
 'criterion': 'gini'}

In [17]:
f1_score(y_pred=optimal_clf.best_estimator_.predict(X_test), y_true=y_test, average='macro')

0.533911925425908